In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
# 현재경로 확인
os.getcwd()

'C:\\실습자료\\part4'

In [5]:
# 데이터 불러오기
data = pd.read_csv("./data/otto_train.csv") # Product Category
data.head() # 데이터 확인

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [6]:
'''
id: 고유 아이디
feat_1 ~ feat_93: 설명변수
target: 타겟변수 (1~9)
'''

'\nid: 고유 아이디\nfeat_1 ~ feat_93: 설명변수\ntarget: 타겟변수 (1~9)\n'

In [7]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 95


## 의미가 없다고 판단되는 변수 제거

In [8]:
data = data.drop(['id'], axis = 1) # id 제거

## 타겟 변수의 문자열을 숫자로 변환

In [9]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [10]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [11]:
# !pip install xgboost
import xgboost as xgb
import time
start = time.time() # 시작 시간 지정
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환
xgb_param = {'max_depth': 10, # 트리 깊이
         'learning_rate': 0.01, # Step Size
         'n_estimators': 100, # Number of trees, 트리 생성 개수
         'objective': 'multi:softmax', # 목적 함수
        'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Accuracy: 76.67 %
Time: 6.35 seconds


In [12]:
xgb_model_predict

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [16]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclass', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Accuracy: 73.57 %
Time: 4.08 seconds


In [17]:
lgb_model.predict(test_x)

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [18]:
# !pip install catboost
import catboost as cb
start = time.time() # 시작 시간 지정
cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

0:	learn: 0.6114702	total: 412ms	remaining: 40.8s
1:	learn: 0.6235304	total: 760ms	remaining: 37.2s
2:	learn: 0.6410246	total: 1.12s	remaining: 36.1s
3:	learn: 0.6410650	total: 1.47s	remaining: 35.3s
4:	learn: 0.6442770	total: 1.81s	remaining: 34.5s
5:	learn: 0.6455901	total: 2.18s	remaining: 34.1s
6:	learn: 0.6468224	total: 2.55s	remaining: 33.8s
7:	learn: 0.6484991	total: 2.88s	remaining: 33.2s
8:	learn: 0.6512666	total: 3.23s	remaining: 32.6s
9:	learn: 0.6491859	total: 3.56s	remaining: 32s
10:	learn: 0.6517716	total: 3.9s	remaining: 31.6s
11:	learn: 0.6526807	total: 4.25s	remaining: 31.2s
12:	learn: 0.6531655	total: 4.59s	remaining: 30.8s
13:	learn: 0.6537918	total: 4.93s	remaining: 30.3s
14:	learn: 0.6566603	total: 5.25s	remaining: 29.8s
15:	learn: 0.6563573	total: 5.6s	remaining: 29.4s
16:	learn: 0.6582966	total: 5.93s	remaining: 29s
17:	learn: 0.6591047	total: 6.27s	remaining: 28.6s
18:	learn: 0.6597309	total: 6.62s	remaining: 28.2s
19:	learn: 0.6603572	total: 7s	remaining: 28s
2

In [20]:
cb_model.predict(test_x)

array([[-0.34047199,  1.18951783,  0.4296545 , ..., -0.17528062,
        -0.02512535, -0.23089952],
       [-0.05653916,  0.45365426,  0.17195659, ...,  0.22287606,
         0.26190632,  0.18447355],
       [-0.31311   , -0.31539546, -0.31876501, ..., -0.29816626,
        -0.238871  , -0.3180434 ],
       ...,
       [ 0.04726952,  0.03794315, -0.17880234, ..., -0.2456077 ,
         0.11886516,  1.54034182],
       [-0.55746055,  1.77202298,  0.97862619, ..., -0.33775734,
        -0.48741853, -0.37792919],
       [-0.31665504,  0.08335556, -0.10060578, ...,  0.67896862,
         1.03198771, -0.14349506]])

In [21]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [22]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

In [23]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [89]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [90]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))

210904.17249451784

In [91]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index,], label = train_y.iloc[random_data_index,]) # 학습 데이터를 LightGBM 모델에 맞게 변환
    lgb_param = {'max_depth': 14, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9615


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


9602
9601
9563
9468
9509
9601
9571
9480
9640


In [92]:
bagging_predict_result

[array([ 531170.32964683,  610437.80671003,  935793.13124286, ...,
         331964.44594363, 1009790.22112937,  451345.9132272 ]),
 array([495196.13568546, 646054.39267815, 986365.56528169, ...,
        331252.38335085, 907530.55639732, 468913.77372686]),
 array([512873.63822953, 654969.90479597, 998836.27669227, ...,
        327405.4302911 , 876943.02927024, 470934.42959936]),
 array([493825.84400474, 636737.39852681, 932594.32632075, ...,
        346914.54108486, 897040.92609115, 464065.61545808]),
 array([ 498631.77754482,  607710.21423254, 1020882.76294162, ...,
         335199.49778924,  936386.22238716,  424294.62431666]),
 array([506096.98682302, 669609.23719954, 940485.52597156, ...,
        345055.0638638 , 918021.10680715, 463670.6100106 ]),
 array([547657.77823348, 642343.41762371, 915100.66250346, ...,
        344643.67265125, 901723.62349688, 455516.83615459]),
 array([ 499624.58304481,  610091.26845662,  967500.51799746, ...,
         359938.28070706, 1006753.92094141,  4

In [93]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [94]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 210563.02109308538


In [95]:
bagging_predict

[511537.7682463254,
 638316.6312678291,
 964783.9053257385,
 1651473.0621924195,
 635458.8364319289,
 369616.1986700068,
 697323.7556415825,
 439239.20639611094,
 463719.21994845115,
 496278.8865646486,
 635330.4595861137,
 380949.3351923268,
 299463.21258238464,
 360208.5291746681,
 342662.0714479795,
 1282739.0039870231,
 365661.43236509623,
 1037815.2680199754,
 316843.57195827994,
 527732.687578006,
 376460.4302308132,
 1858866.039610607,
 664442.000763714,
 532436.038253558,
 499359.45404660434,
 486192.75780494884,
 295520.05474321963,
 249942.02983946315,
 481762.60456522927,
 535161.119001068,
 491196.5262989489,
 467463.33017627697,
 460228.44582321605,
 575697.7774500048,
 376901.7853030611,
 1064716.175740643,
 892564.7722792244,
 526185.5765621655,
 357476.22990572115,
 1547552.4366614297,
 396283.7188693193,
 279266.0073711027,
 513239.44078056526,
 344378.93815549224,
 256820.51854935676,
 242538.0632404439,
 330416.8930207606,
 335083.98714088247,
 354722.9074551059,
 57